# INFO-F-422 - Statistical Foundations of Machine Learning

**Teaching assistants:**<br>
Gian Marco Paldino - gian.marco.paldino@ulb.be<br>
Cédric Simar - cedric.simar@ulb.be<br>
Pascal Tribel - pascal.tribel@ulb.be

**Reference: "Statistical foundations of machine learning: the handbook"** <br>
by **Gianluca Bontempi**, Machine Learning Group, Computer Science Department, ULB

# Practical 3


---

## 1. Introduction

### 1.1 Content of the practical
This practical will address the following topics:
1. Typical **data preprocessing** steps (handling missing values, normalization, dealing with categorical variables).
2. A **vanilla implementation** of the **K-Nearest Neighbors (KNN)** algorithm for regression.
3. Regression with tree-based models: **regression trees** and **Random forests**
5. **Feature importances** in tree-based models.
6. **Scikit-learn pipelines** to combine preprocessing and modeling into a single object.

### 1.2 Data preprocessing overview

Data preprocessing is the foundation of any successful machine learning project. Many Machine Learning models see their performances degraded when trained on poorly preprocessed data. Key steps include:



- **Handling Missing Values**: Missing data can bias results if not addressed. This problem affects many models—such as linear models, support vector machines (SVMs), and neural networks—but it is particularly critical for distance-based algorithms like KNN where missing feature values can disrupt the calculation of distances, leading to inaccurate outcomes. Common strategies for handling missing values include:
  
  - *Dropping* rows/columns with missing data.
  - *Imputing* missing data using simple statistics (mean, median) or more sophisticated methods like **`KNNImputer`**.<br><br>
- **Normalization / Scaling**: Many algorithms assume features are on similar scales. Without proper scaling, features with larger numerical ranges may dominate those with smaller ranges, potentially biasing the model's results. Common scaling strategies include:
  - *Standardization*: Transform features to have zero mean and unit variance.
  - *Min-Max Scaling*: Scale features to a fixed $[0,1]$ range.<br><br>
- **Dealing with Categorical Variables**: Categorical variables must be converted to a numeric representation that preserves meaningful differences. Common strategies for dealing with categorical variables include:
  - *Label Encoding*: Assign an integer to each category (useful if there is an **ordinal** relationship).
  - *One-Hot Encoding (Dummy Encoding)*: Create a new binary column for each category (common for **nominal** features). This approach can cause a significant increase in input dimensionality (see the curse of dimensionality).

### 1.2 K-Nearest Neighbors

The *$K$-Nearest Neighbors* (KNN) algorithm is a **local modeling** approach that infers its prediction from the data points closest (in some metric) to a query. Suppose we have a training set of inputs $\{x_i\}$ with associated labels or targets $\{y_i\}$. For a new query point $q$:

1. **Distance Computation**: Compute the distance (e.g., Euclidean) between $q$ and each training example $x_i$.
2. **Ranking**: Sort or rank the training points by increasing distance to $q$.
3. **Neighborhood Selection**: Identify the subset of the $K$ nearest neighbors $\{x_{[1]}, \dots, x_{[K]}\}$ where $y_{[i]}$ denotes the label of neighbor $x_{[i]}$.
4. **Probability Estimation (Classification)**: Estimate the conditional probability of belonging to class 1, for instance, by

$$
\hat{p}_1(q) \;=\; \frac{1}{K}\sum_{i=1}^{K} y_{[i]},
$$

which is simply the proportion of neighbors labeled “1.” Alternatively, a local linear fit can be used:

$$
\hat{p}_1(q) \;=\; \hat{a}^T q \;+\; \hat{b},
$$

where $\hat{a}$ and $\hat{b}$ are estimated (locally) by least-squares on the neighbors.

5. **Final Prediction**: In classification, if $\hat{p}_1(q) \ge 0.5$, label the query as class 1 (or use majority vote among the $K$ neighbors). For **regression**, one might simply average the targets of the neighbors.

A key hyperparameter here is $K$. A *smaller* $K$ reduces bias but increases variance (the model may overfit local noise), while a *larger* $K$ produces a smoother model at the risk of higher bias. KNN’s performance also heavily depends on the distance metric, especially when features have different scales or include categorical variables (which may require specialized encodings or distance definitions).


### 1.3 Regression trees

Regression trees rely on a tree-based structure of *internal nodes* (where decisions are made) and *terminal nodes* (leaves), which partition the input space into **mutually exclusive** regions, each associated with a simple (often constant) local model. Its construction begins with a **tree growing** phase: starting from the root node that contains all data, we recursively choose a split that best reduces the empirical error. Specifically, for a node $t$ containing $N(t)$ samples, we define

$$
R_{\mathrm{emp}}(t) \;=\; \min_{\alpha_t} \sum_{i=1}^{N(t)} L\bigl(y_i,\;h_t(x_i,\;\alpha_t)\bigr),
$$

where $L$ is typically the squared error $(y - \hat{y})^2$ for regression. Given a potential split $s$ dividing $t$ into children $t_l$ and $t_r$, we consider the decrease in empirical error:

$$
\Delta E(s,\;t) \;=\; R_{\mathrm{emp}}(t)\;-\;\bigl(R_{\mathrm{emp}}(t_l)\;+\;R_{\mathrm{emp}}(t_r)\bigr).
$$

We choose the split $s^*$ maximizing $\Delta E$, partition the dataset accordingly, and repeat the procedure recursively until no further improvement is found or a stopping criterion is met. This exhaustive splitting often yields a **very large** tree that overfits the data.

To address overfitting, a **cost-complexity pruning** procedure is commonly used. Introducing a complexity parameter $\lambda \ge 0$ that penalizes the number of leaf nodes, we define the cost-complexity measure:
     $$
     R_{\lambda}(T) = R_{\mathrm{emp}}(T) + \lambda \, |T|,
     $$
     where $|T|$ is the number of terminal nodes (leaves) in tree $T$.
   - By gradually increasing $\lambda$, we obtain a *sequence* of subtrees:
     $$
     T_{\max} \supset T_{L-1} \supset \dots \supset T_2 \supset T_1.
     $$
     Each has fewer leaves. We consider all admissible subtrees $T_t \subset T$ of the large tree and compute the smallest
     $$
     \lambda_t = \frac{R_{\mathrm{emp}}(T) - R_{\mathrm{emp}}(T_t)}{|T| - |T_t|}
     $$
     that yields a lower cost. We select the subtree that **minimizes** $R_{\lambda}(T)$ (the best balances between empirical error and model complexity).
   - The final tree structure is typically chosen via cross-validation or held-out validation to find the best $\lambda$.

Below is a short snippet demonstrating how to initialize a **DecisionTreeRegressor** in `sklearn` with default parameters (such as `max_depth`, `min_samples_split`, etc.):

```python
from sklearn.tree import DecisionTreeRegressor

tree_reg_example = DecisionTreeRegressor(
    criterion='squared_error',       # default cost for regression
    max_depth=None,                  # no maximum depth
    min_samples_split=2,             
    min_samples_leaf=1,              
    random_state=42                  
)
```
This model is then grown with the standard CART algorithm described above.

For a more visual representation of tree-based models, an animated version of decision trees (classification) can be found here: <http://www.r2d3.us/visual-intro-to-machine-learning-part-1/>

### 1.4 Random Forests

A **Random Forest** (RF) is an ensemble learning technique designed to reduce the variance of decision trees by combining two main ideas:

1. **Bootstrap Sampling** (Bagging): We generate $B$ *bootstrap* datasets, each by sampling (with replacement) from the original training data.
2. **Random Feature Selection** (Feature Bagging): At each split, a *random subset* of $n' < n$ features is chosen, and the best split is found *only among those $n'$ features*.

Hence, each tree $h_b(\mathbf{x}, \alpha_b)$ is built from a *different* resampled dataset **and** uses only a random subset of features at each split. As a result, the trees are more **decorrelated** and often are *not pruned* heavily.

Formally, for a regression task:
$$
  h_{\mathrm{rf}}(\mathbf{x}) = \frac{1}{B} \sum_{b=1}^{B} h_b(\mathbf{x}, \alpha_b),
$$
i.e., the **average** of all tree predictions. (In classification, we take the majority vote.)

#### Why does this reduce variance?

Suppose each tree $h_b$ has variance $\sigma^2$ and they have pairwise correlation $\rho$. The variance of the RF predictor $h_{\mathrm{rf}}$ can be approximated as:
$$
  \mathrm{Var}[h_{\mathrm{rf}}] \approx \frac{(1 - \rho)\,\sigma^2}{B} + \rho\,\sigma^2,
$$
which shows that **increasing** $B$ (the number of trees) reduces the first term, while **lowering** the correlation $\rho$ among trees reduces overall variance. Random feature selection helps reduce $\rho$, because each tree sees a different subset of features.

#### Main Hyperparameters

1. $B$: the number of trees in the forest.  
2. $n'$: the number of randomly selected features at each split (often $\sqrt{n}$ by default in classification).  
3. **Tree parameters**: e.g., maximum depth, minimum samples per leaf, etc.

In practice, random forests typically provide strong performance out of the box and are less sensitive to hyperparameter tuning compared to single trees or more complex models. They also allow computing a useful estimate of **feature importance** by measuring, for each variable, how much it contributes to the cost function improvement across all splits in all trees.

Below is a short snippet demonstrating how to **initialize a RandomForestRegressor** in `sklearn` with default values:
```python
from sklearn.ensemble import RandomForestRegressor

forest_reg_example = RandomForestRegressor(
    n_estimators=100,    # number of trees
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='auto',
    random_state=42
)
```
All of these parameters correspond to the concepts introduced in the sections above.

### 1.5 Feature importances

Most scikit-learn models allows you to estimate how each feature contribues to the final prediction. For example, for tree-based models, both **DecisionTreeRegressor** and **RandomForestRegressor** in scikit-learn store a `feature_importances_` attribute after fitting. This gives an estimate of how much each feature contributed to reducing the split criterion (e.g., MSE). Here’s a quick example using a fitted model:
```python
# Suppose we have a fitted random forest model: forest_reg_example.fit(X, y)
import matplotlib.pyplot as plt
import numpy as np

feature_names = ["Feature1", "Feature2", "Feature3"]  # adapt to your data
importances = forest_reg_example.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot
plt.figure(figsize=(6,4))
plt.title("Feature Importances")
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()
```
The resulting bar plot helps visualize which features were most important for the splits (on average) across the forest. A single decision tree also offers a similar `feature_importances_` vector but typically is less robust than the ensemble measure.

### 1.6 Scikit-learn Pipelines

Pipelines in scikit-learn let us combine **data preprocessing** (e.g., imputation, scaling) and **model training** (e.g., a random forest) into a single workflow object. Below is a simple example:


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Example pipeline: impute missing values with mean, then scale, then fit a linear model
example_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression())
])

print("Demonstration of how a pipeline is defined. No fitting done here.")

Demonstration of how a pipeline is defined. No fitting done here.


---
Sometimes, you may want to incorporate your own transformations in a pipeline. To do this, you can create a custom class that inherits from `BaseEstimator` and `TransformerMixin`. Below is a dummy example that simply adds a constant value to all features.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor

class AddConstantTransformer(BaseEstimator, TransformerMixin):
    """
    Custom transformer that adds a constant value to all features.
    """
    def __init__(self, constant=0.0):
        self.constant = constant

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X + self.constant

custom_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("custom_add_constant", AddConstantTransformer(constant=1.0)),
    ("scaler", StandardScaler()),
    ("regressor", RandomForestRegressor(n_estimators=10, random_state=42))
])

# custom_pipeline.fit(X_train, y_train)
# y_pred = custom_pipeline.predict(X_test)
# print("MSE with custom transformer in pipeline:", mean_squared_error(y_test, y_pred))

---
## 2. Exercise 1: Data Loading, Exploration, and Preprocessing

### 2.1 Instructions
1. Load the well-known **California Housing** regression dataset using the following function:
```python
from sklearn.datasets import fetch_california_housing
```
2. Explore the dataset (distribution, summary statistics).
3. Check for missing values. If the dataset does not contain missing values, artificially introduce some for the purpose of this exercice. Then, handle missing values by:
   - Dropping them (simple approach).
   - Imputing them (mean or median).
   - Using a more sophisticated method (e.g., `KNNImputer`).
4. Normalize the features (using e.g., `StandardScaler`). Remember that, as all processing steps, **scaling is fit on the training set** (not on the entire dataset) to avoid biasing the test set!
5. Preserve original vs. processed versions of the data for comparison.


---
## 3. Exercise 2: Vanilla KNN Implementation

### 3.1 Instructions
1. Split your data into training and test sets (we should have already done that in the previous exercise).
2. Implement a vanilla KNN regressor from scratch (no scikit-learn for this part).
3. Choose a value of $k$ (e.g., 3, 5, 7).
4. Compare predictions with the true values using Mean Squared Error (MSE).

---
## 4. Exercise 3: Regression Trees and Random Forests

### 4.1 Instructions
1. Train a regression tree on the training set.
2. Train a random forest on the training set.
3. Evaluate both models using MSE.
4. Compare their performance with your KNN model.
5. Determine how modifying the `max_depth` (for the regression trees) and `n_estimators` (for the random forests) impacts the performances of the model. Plot these results graphically.
6. Plot the features importance of the best model with the optimal parameters

---
## 5. Exercise 4: Building a Pipeline

### 5.1 Instructions
1. **Define** a pipeline that applies the preprocessing steps (e.g., imputation, scaling) and the best-performing model (from Exercise 3).
2. **Fit** this pipeline on the training data and evaluate on the test data.
3. **Observe** how scikit-learn handles transformations during `fit` and `predict`.


---
---
## 6. Additional exercise: **Empirical bias–variance decomposition on the California Housing Dataset**

In the *synthetic* bias/variance notebooks seen during the lecture, we knew the true function $f(\mathbf x)$, so we could compute the MISE, bias, and variance using Monte Carlo. However, **in real data**, we do *not* know the true function, but we can still estimate an "empirical" bias and variance, and estimate a generalization error (MISE) via cross-validation. In the bias/variance notebook, the MISE was estimated using a Leave-One-Out (LOO) cross-validation. However, for large datasets and/or complex models, a LOO cross-validation can quickly become prohibitively time-consuming. Thus, in practice, using a repeated k-fold cross-validation will drastically reduces the total number of retrains while still offering a robust error estimate.

### 6.1 **Bias–variance decomposition**
First, let's restate the **bias–variance decomposition** seen in the lecture. Suppose our data are generated by:

$$
y = f(\mathbf{x}) + w,
\quad\text{where } w \text{ is zero-mean noise with variance } \sigma_{\mathbf{w}}^2.
$$

We train a model (or "hypothesis") $h(\mathbf{x}; \boldsymbol{\alpha}_N)$ on a dataset of size $N$. We are interested in the **mean squared error (MSE)** of this model’s prediction at a given input $\mathbf{x}$, defined as:

$$
\text{MSE}(\mathbf{x}) = \mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - y\right)^2\right].
$$

Here, the expectation $\mathbb{E}[\cdot]$ is taken over all randomness present—specifically, different training sets (if $\boldsymbol{\alpha}_N$ depends on a randomly drawn dataset) and the noise $w$. Since $y = f(\mathbf{x}) + w$, we can rewrite this as:

$$
\text{MSE}(\mathbf{x}) = \mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - (f(\mathbf{x}) + w)\right)^2\right].
$$

If the hypothesis $h(\mathbf{x}; \boldsymbol{\alpha}_N)$ does not directly depend on the instantaneous noise $w$ at test time, we can separate the noise variance from the model error:

$$
\text{MSE}(\mathbf{x}) =
\underbrace{\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - f(\mathbf{x})\right)^2\right]}_{\text{error from the model itself}}
+ \underbrace{\mathbb{E}[\,w^2\,]}_{\sigma_{\mathbf{w}}^2}.
$$

Focusing specifically on the **model error term**:

$$
\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - f(\mathbf{x})\right)^2\right],
$$

we perform the classical decomposition by adding and subtracting the term $\mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)]$ inside the squared expression:

\begin{aligned}
&\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - f(\mathbf{x})\right)^2\right] \\[1em]
&\quad= \mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - \mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)] + \mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)] - f(\mathbf{x})\right)^2\right] \\[1em]
&\quad= \underbrace{\left(\mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)] - f(\mathbf{x})\right)^2}_{\text{Bias}^2(\mathbf{x})}
+ \underbrace{\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - \mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)]\right)^2\right]}_{\text{Var}(\mathbf{x})} \\[1em]
&\quad\quad+ \underbrace{2\,\mathbb{E}\left[(h - \mathbb{E}[h])(\mathbb{E}[h] - f)\right]}_{=\,0\;\text{by definition}}.
\end{aligned}

In this equation, we use the shorthand $h = h(\mathbf{x}; \boldsymbol{\alpha}_N)$, and note that $\mathbb{E}[h - \mathbb{E}[h]] = 0$, so the cross-term vanishes. Thus, we have the classical bias–variance decomposition:

$$
\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - f(\mathbf{x})\right)^2\right] =
\underbrace{\left(\mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)] - f(\mathbf{x})\right)^2}_{\text{Bias}^2(\mathbf{x})}
+ \underbrace{\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - \mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)]\right)^2\right]}_{\text{Var}(\mathbf{x})}.
$$

Including the irreducible noise $\sigma_{\mathbf{w}}^2 = \mathbb{E}[w^2]$, the full MSE decomposition becomes:

$$
\text{MSE}(\mathbf{x}) =
\underbrace{\left(\mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)] - f(\mathbf{x})\right)^2}_{\text{Bias}^2(\mathbf{x})} +
\underbrace{\mathbb{E}\left[\left(h(\mathbf{x}; \boldsymbol{\alpha}_N) - \mathbb{E}[h(\mathbf{x}; \boldsymbol{\alpha}_N)]\right)^2\right]}_{\text{Var}(\mathbf{x})} +
\underbrace{\sigma_{\mathbf{w}}^2}_{\text{Noise}}.
$$

---

## 6.2 Instructions:

In this exercise, you should:
1. Load the **California Housing** dataset.
2. Define four types of learners:
   - **Constant model**: always predicts the mean of the training labels.
   - **Shallow underfitting tree**: decision tree with depth = 2.
   - **Balanced decision tree**: decision tree with depth = 6.
   - **Overfitting decision tree**: decision tree with depth = 20.
3. Perform **repeated 10-fold cross-validation** to collect multiple predictions per sample.
4. Approximate each model’s:
   - **MSE (CV)**: The standard cross-validation mean squared error (which we will call $\widehat{\text{MISE}}_{\text{kfold}}$).
   - **Bias\(^2\)**: Based on how far the *average prediction* per sample is from the sample's true label.
   - **Variance**: Based on the variability of predictions per sample around that sample’s mean prediction.
   - **MSE (approx)**: the approximation $
\text{MSE} \approx \text{Bias}^2 + \text{Variance}.$
5. Plot and compare all four quantities side by side for each model. Compare the bar plots of these metrics to see how underfitting vs overfitting affect the bias and variance.

**Comment:**

Because real data do not provide a known true function $f$, we approximate it by using each data sample’s **actual** label $y_i$. We can thus use the following approximate bias–variance decomposition:
$$
\text{MSE}(\mathbf{x}) \approx
\underbrace{\left(\overline{\hat{y}(\mathbf{x})} - y(\mathbf{x})\right)^2}_{\text{Bias}^2(\mathbf{x})} \;+\;
\underbrace{\frac{1}{N}\sum_{n=1}^N \Bigl(\hat{y}_n(\mathbf{x}) - \overline{\hat{y}(\mathbf{x})}\Bigr)^2}_{\text{Var}(\mathbf{x})}.
$$
